<a href="https://colab.research.google.com/github/mjavadzadeh11/Crawler/blob/master/TradingView_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install GoogleNews
!pip install newspaper3k;

In [ ]:
from bs4 import BeautifulSoup 
import requests 
import pandas as pd
import datetime as dt
import json
import matplotlib.pyplot as plt
from newspaper import Article
import nltk
nltk.download('punkt')
from textblob import TextBlob

In [3]:
url = "https://www.tradingview.com/key-events/?market=crypto"
source = requests.get(url).text
cf = BeautifulSoup(source , 'lxml')

In [4]:
#getting urls:
links=[]
for link in cf.find_all('a', class_='cardLink-117psRQF'):
  links.append(link['href'])
  #getting img urls:
img_url=[]
for img in cf.find_all('div' , class_='imageWrapper-2Nz2Ij_B ratio2by1-1D2FG29r'):
  img_url.append(img.img['src'])

In [5]:
#content of News : 
df_dict={'Symbol':[] , 'Date':[] , 'Headlines':[], 'Desc':[]}
for content in cf.find_all('div' , class_= 'content-117psRQF'):
  df_dict['Symbol'].append(content.find('span', class_='symbolName-1fSAmU9b').text)
  df_dict['Date'].append(content.find('span',class_='date-1fSAmU9b').text)
  df_dict['Headlines'].append(content.find('div', class_='headline-1fSAmU9b').text)
  df_dict['Desc'].append(content.find('div', class_='description-1fSAmU9b').text)

In [6]:
#change the type of date in tradingview
def norm_date(n):
  ''' change trading view date formate to datetime'''
  
  months_dict={'Jan':1, 'Feb':2,'Mar':3,'Apr':4,'May':5, 'Jun':6, 'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}
  month=n.split(' ')[0]
  month=months_dict[month]
  year=dt.datetime.today().year
  day=int(n.split(' ')[1])
  return dt.date(year,month,day)

df_dict['Date'] = list(map(norm_date,df_dict['Date']))

In [7]:
# Transfer datas into dataFrame
df=pd.DataFrame(df_dict)
df['Links']= links
df['Img_links'] =  img_url


In [ ]:
df.head(2)

In [9]:
for index in range(df.shape[0]):
  try:
    article=Article(df.loc[index,'Links'])
    article.download()
    article.html
    article.parse()
    article.nlp()
    
    
    df.loc[index,'Context'] = article.text
    df.loc[index, 'Summary'] = article.summary
    df.loc[index, 'Keywords'] = ' '.join(article.keywords)
  except:
    df.loc[index,'Context'] = 'Not able to recieve this news'
    df.loc[index, 'Summary'] = 'Could not summarize the context'
    df.loc[index, 'Keywords'] = "No keywords founded"

In [10]:
df.set_index('Date', inplace=True)

In [11]:
df.head(2)

,Symbol,Headlines,Desc,Links,Img_links,Context,Summary,Keywords
Date,,,,,,,,
2021-12-10,MATICUSD,Polygon’s power is growing,Polygon ups its domination game by purchasing ...,https://www.tradingview.com/symbols/MATICUSD/h...,https://s3.tradingview.com/timeline/9-min_3494...,1/8 It's official: #Polygon welcomes @MirProto...,1/8 It's official: #Polygon welcomes @MirProto...,major welcomes events maticusd scaling timelin...
2021-12-10,ETHUSD,Upgrade Activate,Ethereum loses its weekly gains despite an exc...,https://www.tradingview.com/symbols/ETHUSD/his...,https://s3.tradingview.com/timeline/3-min_3494...,Ethereum logs its first weekly loss since Marc...,But it looks like there might be a chink in it...,bitcoin major events musk billion interest tim...


In [12]:
def get_polarity(text):
  return TextBlob(text).sentiment.polarity

In [13]:
title_polarity=df.loc[:,'Headlines'].apply(get_polarity)
desc_polarity=df.loc[:,'Desc'].apply(get_polarity)
df['Polarity']=(title_polarity+desc_polarity)/2